# Fake News classification

В этом треке необходимо научиться предсказывать подлинность новости. В качестве данных вам предстоит работать с текстами новостей с различных информационных порталов. На этих данных вам предстоит обучить модель машинного обучения, предсказывающую является ли новость фейком или нет.

Ссылка на соревнование: [Fake News classification](https://competitions.codalab.org/competitions/26284#learn_the_details-overview)

# XLM-RoBERTa-large

Для решения данной задачи я использовала предобученную модель **XLM-RoBERTa** из библиотеки **Transformers** от **Hugging Face**. Модель была обучена на 100 языках, включая русский, поэтому должна хорошо подойти для классификации русских новостных текстов.

Подробнее о модели можно прочитать по ссылкам:

*   [Unsupervised Cross-lingual Representation Learning at Scale](https://arxiv.org/abs/1911.02116)
*   [xlm-roberta-large](https://huggingface.co/xlm-roberta-large)



## Dependencies

Установим библиотеки от Hugging Face

In [ ]:
!pip install transformers==3.1.0
!pip install -U nlp

     |████████████████████████████████| 890kB 3.3MB/s 
     |████████████████████████████████| 3.0MB 16.2MB/s 
     |████████████████████████████████| 890kB 50.6MB/s 
     |████████████████████████████████| 1.1MB 48.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=35f2de63eb354bfa7d68bb981da9e9150893a86c64e716eb7be5c24fdcd221ae
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.7MB 3.4MB/s 
     |████████████████████████████████| 245kB 55.9MB/s 
     |████████████████████████████████| 17.3MB 262kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


Для работы на TPU установим PyTorch-XLA

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version 20200924  --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5115  100  5115    0     0  26640      0 --:--:-- --:--:-- --:--:-- 26640
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200924 ...
     |████████████████████████████████| 61kB 2.9MB/s 
Uninstalling torch-1.6.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200924-cp36-cp36m-linux_x86_64.whl...
\ [1 files][113.8 MiB/113.8 MiB]                                                
Operation completed over 1 objects/113.8 MiB.                                    


Импортируем необходимые библиотеки

In [ ]:
import os
os.environ['XLA_USE_BF16'] = "1"
os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

import gc
gc.enable()

import numpy as np
import pandas as pd

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader 
from torch.utils.data.distributed import DistributedSampler

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.serialization as xser
import torch_xla.version as xv

import warnings
warnings.filterwarnings("ignore")

Зафиксируем `seed` для воспроизводимости результата

In [ ]:
import random

def my_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

my_seed()

## Model

Загружаем предобученную модель `xlm-roberta-large` и токенизатор

In [ ]:
WRAPPED_MODEL = xmp.MpModelWrapper(AutoModelForSequenceClassification.from_pretrained("xlm-roberta-large"))
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.we

## Data

Считываем данные для обучения

In [ ]:
train_df = pd.read_csv('/content/train.csv', delimiter="\t")

В наших данных всего 2 колонки:  

*   **text** - текст новости
*   **class** - класс новости, принимающий два значения: `true`, если новость настоящая, и `fake` в противном случае



In [ ]:
train_df

,text,class
0,В Синьцзян-Уйгурском автономном районе Китая С...,true
1,Режиссер Дэвид Линч один из создателей телесер...,true
2,Министр регионального развития России Виктор Б...,true
3,На несогласованной акции в поддержку украинско...,true
4,Госдума РФ одобрила в первом чтении законопрое...,true
...,...,...
23878,Пожар на плавучем дебаркадере в центре Москвы ...,true
23879,В Омске возбудили уголовное дело в отношении 6...,true
23880,\n var PROJECTID 80674\n function w d c...,true
23881,Бывший главный тренер московского футбольного ...,true


В колонке **text** имеются пустые значения

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23883 entries, 0 to 23882
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    23878 non-null  object
 1   class   23883 non-null  object
dtypes: object(2)
memory usage: 373.3+ KB


Классы несбалансированы

In [ ]:
train_df['class'].value_counts()

true    19919
fake     3964
Name: class, dtype: int64

Удалим строки с пустым значением **text**, никакой полезной информации в них нет

In [ ]:
train_df.dropna(inplace=True)

In [ ]:
train_df.reset_index(drop=True, inplace=True)

Теперь осталось 23878 примеров в нашем датасете

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23878 entries, 0 to 23877
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    23878 non-null  object
 1   class   23878 non-null  object
dtypes: object(2)
memory usage: 373.2+ KB


In [ ]:
train_df

,text,class
0,В Синьцзян-Уйгурском автономном районе Китая С...,true
1,Режиссер Дэвид Линч один из создателей телесер...,true
2,Министр регионального развития России Виктор Б...,true
3,На несогласованной акции в поддержку украинско...,true
4,Госдума РФ одобрила в первом чтении законопрое...,true
...,...,...
23873,Пожар на плавучем дебаркадере в центре Москвы ...,true
23874,В Омске возбудили уголовное дело в отношении 6...,true
23875,\n var PROJECTID 80674\n function w d c...,true
23876,Бывший главный тренер московского футбольного ...,true


In [ ]:
train_df['class'].value_counts()

true    19914
fake     3964
Name: class, dtype: int64

Преобразуем метки классов в **0** и **1**

In [ ]:
from sklearn.preprocessing import LabelBinarizer
binarizer = LabelBinarizer()

train_df['class'] = binarizer.fit_transform(train_df['class'])

In [ ]:
train_df

,text,class
0,В Синьцзян-Уйгурском автономном районе Китая С...,1
1,Режиссер Дэвид Линч один из создателей телесер...,1
2,Министр регионального развития России Виктор Б...,1
3,На несогласованной акции в поддержку украинско...,1
4,Госдума РФ одобрила в первом чтении законопрое...,1
...,...,...
23873,Пожар на плавучем дебаркадере в центре Москвы ...,1
23874,В Омске возбудили уголовное дело в отношении 6...,1
23875,\n var PROJECTID 80674\n function w d c...,1
23876,Бывший главный тренер московского футбольного ...,1


In [ ]:
train_df['class'].value_counts()

1    19914
0     3964
Name: class, dtype: int64

Разделим наш датасет на две части, выделив для валидации 1024 примера (фиксируем random state для одинакового разбиения при перезапуске ноутбука)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(train_df, test_size=1024, random_state=42)

In [ ]:
train_df

,text,class
21835,Фактическое отсутствие венчурного финансирован...,1
2923,В Липецкой области у мужчины при попытке устро...,1
6888,В результате обрушения стены строящегося здани...,1
8149,Москва не видит причин для ввода новых экономи...,1
4189,Боевики группировки Исламское государство ИГ к...,1
...,...,...
21575,Об организации в противовес западному Хеллоуин...,0
5390,"Сирия нашла нового покупателя на свою черешню,...",1
860,"Закон, определяющий территорию самопровозглаше...",1
15795,Президент России Дмитрий Медведев одобрил пред...,1


In [ ]:
valid_df

,text,class
12635,В ЕС считают неполным отвод вооружений украинс...,1
19667,В Подмосковье убит бывший руководитель аэропор...,1
20987,\n function catchOutboundClicklink \n li...,1
11961,Турецкая экономическая газета Dunya со ссылкой...,1
4413,Все высшие должностные лица Российской Федерац...,1
...,...,...
10696,Белоруссия ведет переговоры с Ираном о поставк...,1
7558,В прошлом внутренности комет были жидкими - та...,1
3213,На фоне обвинений звучащих в Конгрессе США в а...,0
11259,Двое граждан Германии похищены на востоке Лива...,1


Теперь прочитаем тестовый набор данных

In [ ]:
test_df = pd.read_csv('/content/test.csv', delimiter="\t")

In [ ]:
test_df

,text,class
0,Британское министерство информации приняло дек...,NaN
1,"Альбом ""Chief"" кантри-исполнителя Эрика Черча ...",NaN
2,Новый вид микрокредитных организаций появился ...,NaN
3,Один из крупнейших торговых комплексов Москвы ...,NaN
4,Министерство обороны Белоруссии приостановила ...,NaN
...,...,...
6019,Представители интернет-сообщества попросили Го...,NaN
6020,Московские кришнаиты как и было запланировано ...,NaN
6021,Компания Fox Broadcasting договорились с опера...,NaN
6022,Гражданка Великобритании Джорджина Чайлдс задо...,NaN


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6024 entries, 0 to 6023
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    6024 non-null   object 
 1   class   0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 94.2+ KB


Для обучения колонка **class** нам будет не нужна

In [ ]:
test_df.drop(columns=['class'], inplace=True)

In [ ]:
test_df

,text
0,Британское министерство информации приняло дек...
1,"Альбом ""Chief"" кантри-исполнителя Эрика Черча ..."
2,Новый вид микрокредитных организаций появился ...
3,Один из крупнейших торговых комплексов Москвы ...
4,Министерство обороны Белоруссии приостановила ...
...,...
6019,Представители интернет-сообщества попросили Го...
6020,Московские кришнаиты как и было запланировано ...
6021,Компания Fox Broadcasting договорились с опера...
6022,Гражданка Великобритании Джорджина Чайлдс задо...


## Datasets

С помощью токенизатора преобразуем наши тексты в вектора `input_ids` одной длины, а также укажем необходимый для нашей модели формат тензоров `torch`

In [ ]:
import nlp

In [ ]:
MAX_LEN = 256

def convert_to_features(batch):

    encodings = tokenizer.batch_encode_plus(batch['text'], 
                                            add_special_tokens=True,
                                            pad_to_max_length=True,
                                            truncation=True,
                                            max_length=MAX_LEN)
    return encodings

train_dataset = nlp.Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(convert_to_features, batched=True)
train_dataset.set_format("torch", columns=['input_ids', 'attention_mask', 'class'])

valid_dataset = nlp.Dataset.from_pandas(valid_df)
valid_dataset = valid_dataset.map(convert_to_features, batched=True) 
valid_dataset.set_format("torch", columns=['input_ids', 'attention_mask', 'class'])

test_dataset = nlp.Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(convert_to_features, batched=True) 
test_dataset.set_format("torch", columns=['input_ids', 'attention_mask'])

## Train & Submit

### Metrics

Во время обучения будем следить за значениями функции потерь и метрикой f1

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
class AverageMeter(object):
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {avg' + self.fmt + '}'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

def F1_score(outputs, targets):
    with torch.no_grad():
        return f1_score(targets, outputs.argmax(dim=-1), average='macro') 

### Training

Опишем функцию для обучения

In [ ]:
def train_loop_fn(train_loader, model, optimizer, device, epoch=None):

    model.train()
    
    losses = AverageMeter('Loss:', ':.4e')
    top1 = AverageMeter('F1:', ':.2f')
    progress = ProgressMeter(
        len(train_loader),
        [losses, top1],
        prefix="[xla:{}]  Train:  Epoch: [{}]".format(xm.get_ordinal(), epoch + 1)
    )    

    for i, data in enumerate(train_loader):

        input_ids = data["input_ids"].to(device, dtype=torch.long)
        attention_mask = data["attention_mask"].to(device, dtype=torch.long)
        labels = data["class"].to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            labels = labels
        )

        loss = outputs[0] 
        loss.backward()
        xm.optimizer_step(optimizer)

        f1 = F1_score(outputs[1].cpu(), labels.cpu())
        losses.update(loss.item(), input_ids.size(0))
        top1.update(f1.item(), input_ids.size(0))

        if i == len(train_loader) - 1: 
            progress.display(i)

    del loss
    del f1
    del outputs
    del input_ids
    del attention_mask
    del labels
    gc.collect()

### Evaluation

Опишем функцию для валидации

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
def eval_loop_fn(test_data_loader, model, device):

    model.eval()

    with torch.no_grad():

        test_preds = []
        targets = []
        val_loss = 0

        for i, data in enumerate(test_data_loader):

            input_ids = data["input_ids"].to(device, dtype=torch.long)
            attention_mask = data["attention_mask"].to(device, dtype=torch.long)
            labels = data["class"].to(device, dtype=torch.float)

            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
                labels = labels
            )

            val_loss += outputs[0].item()
            test_preds.extend(torch.argmax(outputs[1], -1).cpu().detach().tolist())
            targets.extend(data["class"].detach().tolist())  
        
    xm.master_print(f'Val loss: {val_loss / len(test_data_loader)}')
    xm.master_print(f'F1 score: {f1_score(targets, test_preds, average="macro")}')
    xm.master_print('Confusion matrix:')
    xm.master_print(f'{confusion_matrix(targets, test_preds)}')

    del outputs
    del targets 
    del test_preds
    del input_ids
    del attention_mask
    gc.collect()

### Test

Опишем функцию, которая будет получать предсказания для тестового датасета, преобразовывать их обратно в **true**/**fake** и записывать в колонку `class` исходного файла `test.csv` под именем `submission_epoch.csv`

In [ ]:
sample_submission = pd.read_csv('/content/test.csv', delimiter="\t")

In [ ]:
def test_loop_fn(test_data_loader, model, device, epoch=0):

    model.eval()

    with torch.no_grad():

        test_preds = []

        for i, data in enumerate(test_data_loader):

            input_ids = data["input_ids"].to(device, dtype=torch.long)
            attention_mask = data["attention_mask"].to(device, dtype=torch.long)

            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
            )[0]

            test_preds.extend(torch.argmax(outputs, -1).cpu().detach().tolist())  

    sample_submission['class'] = binarizer.inverse_transform(np.array(test_preds))
    sample_submission.to_csv(f'submission_{epoch + 1}.csv', sep="\t")

    del outputs
    del test_preds
    del input_ids
    del attention_mask
    gc.collect()

### Run

Укажем параметры обучения

In [ ]:
my_seed(42)

TRAIN_BATCH_SIZE = 8 * xm.xrt_world_size()
VALID_BATCH_SIZE = 8 * xm.xrt_world_size()
TEST_BATCH_SIZE = 8
EPOCHS = 5
LR = 5e-5 * xm.xrt_world_size() 

Определим загрузчики данных для каждого датасета и опишем основной цикл обучения. После каждой эпохи будем оценивать качество на валидационном датасете и делать предсказания для тестового.

In [ ]:
def _run():

    # Train
    xm.master_print('Starting Run ...')
    train_sampler = DistributedSampler(
        train_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True 
    )    
    train_data_loader = DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        sampler=train_sampler,
        drop_last=True,
        num_workers=0,
    )
    xm.master_print('Train Loader Created.')
    
    # Valid   
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        drop_last=False,
        num_workers=0,
        shuffle=False
    )
    xm.master_print('Valid Loader Created.')

    # Test
    test_data_loader = DataLoader(
        test_dataset, 
        batch_size=TEST_BATCH_SIZE,
        drop_last=False,
        num_workers=0,
        shuffle=False,
    )
    xm.master_print('Test Loader Created.')
    
    num_train_steps = int(len(train_df) / TRAIN_BATCH_SIZE / xm.xrt_world_size())
    device = xm.xla_device()
    model = WRAPPED_MODEL.to(device)
    xm.master_print('Done Model Loading.')
    optimizer = AdamW(
        model.parameters(),
        lr=LR, 
    )
    xm.master_print(f'Num Train Steps= {num_train_steps}, XRT World Size= {xm.xrt_world_size()}.')

    for epoch in range(EPOCHS):

        train_sampler.set_epoch(epoch)
        
        para_loader = pl.ParallelLoader(train_data_loader, [device])
        xm.master_print('Parallel Loader Created. Training ...')
        train_loop_fn(para_loader.per_device_loader(device),
                      model,  
                      optimizer, 
                      device,  
                      epoch
                     )        
        xm.master_print("Finished training epoch {}".format(epoch + 1))
            
        para_loader = pl.ParallelLoader(valid_data_loader, [device])
        xm.master_print('Parallel Loader Created. Validating ...')
        eval_loop_fn(para_loader.per_device_loader(device), 
                     model,  
                     device
                    )
        
        para_loader = pl.ParallelLoader(test_data_loader, [device])
        xm.master_print('Parallel Loader Created. Testing ...')
        test_loop_fn(para_loader.per_device_loader(device), 
                    model,  
                    device,
                    epoch 
                    )
        xm.master_print('Predicted.')

        # При необходимости можно сохранить веса модели
        # if epoch == EPOCHS-1:
        #     xm.save(model.state_dict(), f'XLM_R_{epoch + 1}.pt')
        #     xm.master_print('Model Saved.')

Запускаем обучение

In [ ]:
def _mp_fn(rank, flags):
    _run()

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

Starting Run ...
Train Loader Created.
Valid Loader Created.
Test Loader Created.
Done Model Loading.
Num Train Steps= 357, XRT World Size= 8.
Parallel Loader Created. Training ...
[xla:3]  Train:  Epoch: [1][356/357]	Loss: 6.3044e-02	F1: 0.94
[xla:0]  Train:  Epoch: [1][356/357]	Loss: 4.9686e-02	F1: 0.95
[xla:1]  Train:  Epoch: [1][356/357]	Loss: 6.0571e-02	F1: 0.95
[xla:5]  Train:  Epoch: [1][356/357]	Loss: 5.9557e-02	F1: 0.95
[xla:4]  Train:  Epoch: [1][356/357]	Loss: 5.5347e-02	F1: 0.95
[xla:6]  Train:  Epoch: [1][356/357]	Loss: 6.4242e-02	F1: 0.94
[xla:2]  Train:  Epoch: [1][356/357]	Loss: 6.8966e-02	F1: 0.94
[xla:7]  Train:  Epoch: [1][356/357]	Loss: 5.5922e-02	F1: 0.95
Finished training epoch 1
Parallel Loader Created. Validating ...
Val loss: 0.013354778289794922
F1 score: 0.9928104024489566
Confusion matrix:
[[164   1]
 [  3 856]]
Parallel Loader Created. Testing ...
Predicted.
Parallel Loader Created. Training ...
[xla:5]  Train:  Epoch: [2][356/357]	Loss: 4.3562e-03	F1: 1.00

Файл `submission_5.csv` как раз соответствует результату f1=1.0